# Step 2: Model Building
Using the sensor data sets explored and constructed in the `1_data_ingestion_and_preparation.ipynb` Jupyter notebook, this notebook loads the data from the Azure Blob container and builds a LSTM network for scenerio described at [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) to predict remaining useful life of aircraft engines. We then store the model for deployment in an Azure web service. We will prepare and build the web service in the `Code/3_operationalization.ipynb` Jupyter notebook.

In [64]:
import keras

In [65]:
# import the libraries
import h5py
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib
import glob
import pickle
import re

# Setup the pyspark environment
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
spark = SparkSession.builder.getOrCreate()

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from sklearn import datasets
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline

# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger

# For Azure blob storage access
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

In [ ]:
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.predictivemaintenanceforpm.modelbuildingandevaluation','true')

## Load feature data set

We have previously created the labeled data set in the `Code\1_Data Ingestion and Preparation.ipynb` Jupyter notebook. Since the Azure Blob storage account name and account key are not passed between notebooks, you'll need your credentials here again.

In [66]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "<your blob storage account name>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "<your blob storage account key>"

#-------------------------------------------------------------------------------------------
# The data from the Data Ingestion and Preparation notebook is stored in the sensordata ingestion container.
CONTAINER_NAME = "sensordataingestionpm"

# Connect to your blob service     
az_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# We will store and read each of these data sets in blob storage in an 
# Azure Storage Container on your Azure subscription.
# See https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md
# for details.

# This is the final feature data file.
TRAIN_DATA = 'PM_train_files.parquet'
TEST_DATA = 'PM_test_files.parquet'

Load the data and dump a short summary of the resulting DataFrame.

In [67]:
# load the previous created final dataset into the workspace
# create a local path where we store results
if not os.path.exists(TRAIN_DATA):
    os.makedirs(TRAIN_DATA)
    print('DONE creating a local directory!')

# download the entire parquet result folder to local path for a new run 
for blob in az_blob_service.list_blobs(CONTAINER_NAME):
    if TRAIN_DATA in blob.name:
        local_file = os.path.join(TRAIN_DATA, os.path.basename(blob.name))
        az_blob_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

train_df = spark.read.parquet(TRAIN_DATA)
train_df = train_df.toPandas()
train_df.head(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.000000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.002770
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.005540
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.008310
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.011080
5,1,6,0.252874,0.416667,0.0,0.0,0.268072,0.292784,0.272113,0.0,...,0.0,0.250000,0.0,0.0,0.651163,0.652720,186,0,0,0.013850
6,1,7,0.557471,0.583333,0.0,0.0,0.382530,0.463920,0.261985,0.0,...,0.0,0.333333,0.0,0.0,0.744186,0.667219,185,0,0,0.016620
7,1,8,0.304598,0.750000,0.0,0.0,0.406627,0.259865,0.316003,0.0,...,0.0,0.250000,0.0,0.0,0.643411,0.574979,184,0,0,0.019391
8,1,9,0.545977,0.583333,0.0,0.0,0.274096,0.434707,0.211850,0.0,...,0.0,0.333333,0.0,0.0,0.705426,0.707539,183,0,0,0.022161
9,1,10,0.310345,0.583333,0.0,0.0,0.150602,0.440375,0.307394,0.0,...,0.0,0.416667,0.0,0.0,0.627907,0.794256,182,0,0,0.024931


In [68]:
type(train_df)

pandas.core.frame.DataFrame

In [69]:
# load the previous created final dataset into the workspace
# create a local path where we store results
if not os.path.exists(TEST_DATA):
    os.makedirs(TEST_DATA)
    print('DONE creating a local directory!')

# download the entire parquet result folder to local path for a new run 
for blob in az_blob_service.list_blobs(CONTAINER_NAME):
    if TEST_DATA in blob.name:
        local_file = os.path.join(TEST_DATA, os.path.basename(blob.name))
        az_blob_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)
        
test_df = spark.read.parquet(TEST_DATA)
test_df = test_df.toPandas()
test_df.head(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,71,38,0.287356,0.416667,0.0,0.0,0.388554,0.370395,0.296759,0.0,...,0.0,0.166667,0.0,0.0,0.550388,0.615300,0.102493,148,0,0
1,71,39,0.448276,0.666667,0.0,0.0,0.355422,0.342272,0.257427,0.0,...,0.0,0.416667,0.0,0.0,0.581395,0.684065,0.105263,147,0,0
2,71,40,0.396552,0.666667,0.0,0.0,0.286145,0.415522,0.358035,0.0,...,0.0,0.416667,0.0,0.0,0.666667,0.665010,0.108033,146,0,0
3,71,41,0.477011,0.916667,0.0,0.0,0.231928,0.368651,0.371033,0.0,...,0.0,0.250000,0.0,0.0,0.759690,0.730599,0.110803,145,0,0
4,71,42,0.494253,0.416667,0.0,0.0,0.319277,0.315239,0.274983,0.0,...,0.0,0.250000,0.0,0.0,0.674419,0.716101,0.113573,144,0,0
5,71,43,0.350575,0.416667,0.0,0.0,0.262048,0.419228,0.139264,0.0,...,0.0,0.250000,0.0,0.0,0.449612,0.711130,0.116343,143,0,0
6,71,44,0.729885,0.666667,0.0,0.0,0.427711,0.421844,0.346725,0.0,...,0.0,0.500000,0.0,0.0,0.689922,0.560895,0.119114,142,0,0
7,71,45,0.362069,0.833333,0.0,0.0,0.424699,0.363418,0.426063,0.0,...,0.0,0.333333,0.0,0.0,0.519380,0.782657,0.121884,141,0,0
8,71,46,0.643678,0.916667,0.0,0.0,0.289157,0.178984,0.362086,0.0,...,0.0,0.166667,0.0,0.0,0.573643,0.858879,0.124654,140,0,0
9,71,47,0.574713,0.250000,0.0,0.0,0.228916,0.429693,0.449527,0.0,...,0.0,0.250000,0.0,0.0,0.519380,0.728390,0.127424,139,0,0


In [70]:
type(test_df)

pandas.core.frame.DataFrame

## Modelling

The traditional predictive maintenance machine learning models are based on feature engineering which is manual construction of right features using domain expertise and similar methods. This usually makes these models hard to reuse since feature engineering is specific to the problem scenario and the available data which varies from one business to the other. Perhaps the most attractive part of applying deep learning in the predictive maintenance domain is the fact that these networks can automatically extract the right features from the data, eliminating the need for manual feature engineering.

When using LSTMs in the time-series domain, one important parameter to pick is the sequence length which is the window for LSTMs to look back. This may be viewed as similar to picking window_size = 5 cycles for calculating the rolling features in the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) which are rolling mean and rolling standard deviation for 21 sensor values. The idea of using LSTMs is to let the model extract abstract features out of the sequence of sensor values in the window rather than engineering those manually. The expectation is that if there is a pattern in these sensor values within the window prior to failure, the pattern should be encoded by the LSTM.

One critical advantage of LSTMs is their ability to remember from long-term sequences (window sizes) which is hard to achieve by traditional feature engineering. For example, computing rolling averages over a window size of 50 cycles may lead to loss of information due to smoothing and abstracting of values over such a long period, istead, using all 50 values as input may provide better results. While feature engineering over large window sizes may not make sense, LSTMs are able to use larger window sizes and use all the information in the window as input. Below, we illustrate the approach.

In [71]:
# pick a large window size of 50 cycles
sequence_length = 50

[Keras LSTM](https://keras.io/layers/recurrent/) layers expect an input in the shape of a numpy array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step. 

In [72]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [73]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
input_features = sensor_cols + sequence_cols
input_features
sequence_cols.extend(sensor_cols)

In [74]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

In [76]:
# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

In [58]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [59]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(36262, 1)

## LSTM Network

Next, we build a deep network. The first layer is a LSTM layer with 100 units followed by another LSTM layer with 50 units. Dropout is also applied after each LSTM layer to control overfitting. Final layer is a Dense output layer with single unit and sigmoid activation since this is a binary classification problem.

In [60]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 50, 100)           50400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 80,651
Trainable params: 80,651
Non-trainable params: 0
_________________________________________________________________
None


In [79]:
%%time
# fit the network
model.fit(seq_array, label_array, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

In [20]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

15631/15631 [==============================] - 7s 430us/step
Accurracy: 0.9618066698703616


In [21]:
# make predictions and compute confusion matrix
y_pred = model.predict_classes(seq_array,verbose=1, batch_size=200)
y_true = label_array
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

15631/15631 [==============================] - 7s 419us/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[12093,   502],
       [   95,  2941]])

In [22]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

precision =  0.85419692129 
 recall =  0.968708827404


Next, we look at the performance on the test data. Only the last cycle data for each engine id in the test data is kept for testing purposes. In order to compare the results to the template, we pick the last sequence for each id in the test data.

In [23]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(100, 50, 25)

In [24]:
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]

In [25]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

(100, 1)

In [26]:
print(seq_array_test_last.shape)
print(label_array_test_last.shape)

(100, 50, 25)
(100, 1)


In [27]:
# test metrics
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

Accurracy: 0.92


In [28]:
# make predictions and compute confusion matrix
y_pred_test = model.predict_classes(seq_array_test_last)
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[ 0,  4],
       [ 4, 92]])

In [29]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )

Precision:  0.958333333333 
 Recall:  0.958333333333 
 F1-score: 0.958333333333


## Saving/loading the model  

In [30]:
# saving a copy of the file
model_cp = model

In [31]:
model_cp

### Save and test the loaded model locally first

In [32]:
# Save the model for operationalization: https://machinelearningmastery.com/save-load-keras-deep-learning-models/
import os
import h5py
from sklearn import datasets 
 
# save model
# serialize model to JSON
model_json = model.to_json()
with open("modellstm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modellstm.h5")
print("Model saved")

Model saved


In [33]:
from keras.models import model_from_json

# load json and create model
json_file = open('modellstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modellstm.h5")
print("Model loaded")

Model loaded


In [34]:
score = loaded_model.predict_proba(seq_array,verbose=1)
print(score.shape)
print(score)

15631/15631 [==============================] - 9s 572us/step
(15631, 1)
[[ 0.00263677]
 [ 0.0027143 ]
 [ 0.00279908]
 ..., 
 [ 0.98669761]
 [ 0.98698622]
 [ 0.98730135]]


### Save the files to the shared folder for operationalization
https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files

#### Ubuntu Linux
/home/<username>/ then navigate to .azureml/share/<exp_acct_name>/<workspace_name>/<proj_name>/

In [35]:
with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.json', 'wt') as json_file:
    json_file.write(model_json)
    print("json file written shared folder")
    json_file.close()

json file written shared folder


In [36]:
model_cp.save_weights(os.path.join(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'], 'modellstm.h5'))

## Conclusion
In the next notebook `Code/3_operationalization.ipynb` Jupyter notebook we will create the functions needed to operationalize and deploy any model to get realtime predictions.